<a href="https://colab.research.google.com/github/makanadani/vetbot/blob/main/vetbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# Instala a ferramenta de IA Generativa do Gemini

!pip install -q google.genai

# Configura a API Key do Google Gemini

import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [28]:
# Instala framework de agentes do Google

!pip install -q google-adk

# Importa ferramentas necessárias para construção do agente

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [30]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:

    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()

    # Cria uma nova sessão
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""

    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [31]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [41]:
#################################################
# --------- Agente 1: VetBot BUSCADOR --------- #
#################################################

def vetbot_buscador(sintoma, data_de_hoje):
  buscador = Agent(
      name="vetbot_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca informações no Google",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa veterinária, treinado para buscar informações educativas sobre saúde e comportamento de
      cães e gatos no Google (google_search) relacionados ao tópico abaixo. Separe 10 fontes mais relevantes do último ano e priorize
      informações de fontes confiáveis e reconhecidas na medicina veterinária, como universidades e associações, nacionais e internacionais.
      Sempre ressalte que você não substitui a consulta com um médico veterinário."""
  )
  entrada_vetbot_buscador = f"Sintoma do Animal: {sintoma}\nData de hoje: {data_de_hoje}"
  informacoes_gerais = call_agent(buscador, entrada_vetbot_buscador)
  return informacoes_gerais

In [42]:
#################################################
# -------- Agente 2: VetBot FILTRADOR -------- #
#################################################
def vetbot_filtrador(sintoma, informacoes_buscadas):
    filtrador = Agent(
        name="vetbot_filtrador",
        model="gemini-2.0-flash",
        description="Agente que filtra as informações",
        tools=[google_search],
        instruction="""
        Você é um assistente científico, especializado em estudos de cães e gatos. Com base nas lista de fontes encontradas, você deve usar a
        ferramenta de busca do Google (google_search) para embasar as informações encontradas com mais detalhes. Descarte informações de blogs
        pessoais, fóruns e sites sem fontes científicas das informações.
        """
    )
    entrada_vetbot_filtrador = f"Sintoma do Animal:{sintoma}\nInformações buscadas: {informacoes_buscadas}"
    informacoes_filtradas = call_agent(filtrador, entrada_vetbot_filtrador)
    return informacoes_filtradas

In [49]:
##########################################
# ------ Agente 3: VetBot REVISOR ------ #
##########################################
def vetbot_revisor(sintoma, resposta_gerada):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        description="Agente revisor de informações científicas veterinárias.",
        instruction="""
            Você é um veterinário, especializado em atendimento de cães e gatos. Como tal, você sabe a importância de não realizar prescrições ou sugerir
            tratamentos sem a realização de uma consulta com médico veterinário especializado. Dessa forma, revise o texto gerado para excluir quaisquer
            informações que possam configurar prescrição ou orientação sobre tratamentos, deixando somente o que for relevante para um tutor cuidar bem do
            seu cão ou gato, mantendo o que for importante para que ele entenda sobre o comportamento da espécie e os cuidados mais recomendados. Reforce
            que você não substitui o atendimento médico veterinário especializado e oriente-o sobre os riscos de não buscar este tipo de atendimento
            em cada caso relatado.
            """
    )
    entrada_vetbot_revisor = f"Sintoma do Animal: {sintoma}\nRascunho: {resposta_gerada}"
    resposta_final = call_agent(revisor, entrada_vetbot_revisor)
    return resposta_final

In [50]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o VetBot 🚀")

# Obtém o sintoma apresentado pelo animal
sintoma = input("❓ Por favor, digite o SINTOMA que o seu animal apresenta: ")

# Insere lógica do sistema de agentes
if not sintoma:
    print("Você esqueceu de digitar o sintoma!")
else:
    print(f"Obrigado! Vamos buscar o que há de mais recente sobre {sintoma}")

    informacoes_sintoma = vetbot_buscador(sintoma, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(informacoes_sintoma))
    print("--------------------------------------------------------------")

    informacoes_embasadas = vetbot_filtrador(sintoma, informacoes_sintoma)
    print("\n--- 📝 Resultado do Agente 2 (Filtrador) ---\n")
    display(to_markdown(informacoes_embasadas))
    print("--------------------------------------------------------------")

    resposta_embasada = vetbot_revisor(sintoma, informacoes_embasadas)
    print("\n--- 📝 Resultado do Agente 3 (Revisor) ---\n")
    display(to_markdown(resposta_embasada))
    print("--------------------------------------------------------------")

🚀 Iniciando o VetBot 🚀
❓ Por favor, digite o SINTOMA que o seu animal apresenta: vomito
Obrigado! Vamos buscar o que há de mais recente sobre vomito

--- 📝 Resultado do Agente 1 (Buscador) ---



> Compreendo. Para te ajudar a entender melhor os possíveis causas e o que fazer em caso de vômito em cães e gatos, vou realizar uma pesquisa no Google buscando informações educativas sobre saúde e comportamento, priorizando fontes confiáveis e recentes (último ano). Após a pesquisa, vou apresentar as informações encontradas, ressaltando que elas não substituem a consulta com um médico veterinário.
> 
> 
> Olá! O vômito em cães e gatos pode ser um sintoma de diversas condições, desde algo simples como má digestão até problemas mais sérios como infecções ou doenças crônicas. É importante observar o seu animal de estimação com atenção para identificar a causa do vômito e saber quando procurar ajuda veterinária.
> 
> **Causas Comuns de Vômito em Cães e Gatos:**
> 
> *   **Indigestão:** Ingestão rápida de comida, excesso de alimento ou alimentos estragados, mudanças na dieta, intolerância alimentar.
> *   **Corpos estranhos:** Ingestão de objetos como brinquedos, ossos, plantas, fios, elásticos, etc., que podem causar irritação ou obstrução.
> *   **Infecções:** Vírus, bactérias ou parasitas podem causar inflamação no estômago e intestinos.
> *   **Doenças:** Problemas gastrointestinais, renais, hepáticos, pancreáticos, doenças endócrinas (diabetes, hipertireoidismo), tumores, etc.
> *   **Intoxicação:** Ingestão de substâncias tóxicas como produtos de limpeza, medicamentos, plantas venenosas, chocolate, etc.
> *   **Alergias:** Reações alérgicas a alimentos, medicamentos ou substâncias ambientais.
> *   **Estresse:** Mudanças na rotina, viagens, ansiedade, medo.
> *   **Bolas de pelo (em gatos):** Acúmulo de pelos no estômago devido à lambedura.
> 
> **Quando se Preocupar e Procurar um Veterinário:**
> 
> *   **Vômito persistente ou frequente:** Mais de duas vezes ao dia ou por mais de 24 horas.
> *   **Presença de sangue no vômito:** Pode indicar sangramento interno, úlceras ou ingestão de corpos estranhos.
> *   **Vômito acompanhado de outros sintomas:** Diarreia, perda de apetite, letargia (cansaço excessivo), febre, dor abdominal, desidratação, mudanças no comportamento.
> *   **Suspeita de ingestão de substância tóxica ou corpo estranho.**
> *   **Se o animal for filhote ou idoso.**
> *   **Se o animal tiver alguma doença preexistente ou estiver tomando alguma medicação.**
> * Tentativas de vomitar sem sucesso, principalmente em cães, podem indicar torção gástrica, uma emergência veterinária.
> 
> **O que Fazer em Caso de Vômito:**
> 
> 1.  **Observe o vômito:** Anote a frequência, o aspecto (cor, consistência, conteúdo) e se há outros sintomas.
> 2.  **Retire a comida:** Suspenda a alimentação por algumas horas (12-24 horas para cães e gatos adultos, menos tempo para filhotes), mas sempre mantenha água fresca à disposição para evitar desidratação.
> 3.  **Ofereça água em pequenas quantidades:** Se o animal estiver desidratado ou com dificuldade de reter líquidos, ofereça pequenas porções de água com frequência. Água de coco também pode ajudar.
> 4.  **Dieta leve:** Após o período de jejum, introduza gradualmente uma dieta leve e de fácil digestão, como frango cozido desfiado com arroz branco cozido sem tempero.
> 5.  **Higiene:** Limpe o local onde o animal vomitou para evitar a proliferação de bactérias e odores.
> 
> **O que NÃO Fazer:**
> 
> *   **Não medique o animal por conta própria:** Muitos medicamentos humanos são tóxicos para cães e gatos.
> *   **Não force o animal a comer:** Se ele não estiver com apetite, espere até que ele se sinta melhor.
> *   **Não ignore os sinais de alerta:** Se o vômito persistir ou for acompanhado de outros sintomas, procure um veterinário.
> 
> **Observações Importantes:**
> 
> *   A cor do vômito pode dar pistas sobre a causa:
>     *   **Amarelo ou verde:** Presença de bile, geralmente indica estômago vazio.
>     *   **Branco:** Pode indicar excesso de gás no estômago, indigestão ou inflamação.
>     *   **Marrom:** Pode ser comida não digerida ou, em casos mais graves, fezes (coprofagia).
>     *   **Vermelho:** Presença de sangue fresco.
>     *   **Preto:** Sangue digerido, indica sangramento no trato gastrointestinal superior.
> *   É importante diferenciar vômito de regurgitação. A regurgitação é um processo passivo, sem esforço abdominal, e geralmente envolve comida não digerida. O vômito é um processo ativo, com contrações musculares e náuseas.
> 
> Lembre-se que essas informações são apenas um guia. **Em caso de vômito, consulte sempre um médico veterinário para um diagnóstico preciso e tratamento adequado.**
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Filtrador) ---



> Entendi. Para complementar as informações sobre vômito em cães e gatos, realizei algumas pesquisas no Google para trazer informações mais específicas e atualizadas.
> 
> 
> Com base nas informações encontradas, aqui estão alguns pontos adicionais e complementares sobre o vômito em cães e gatos:
> 
> **Causas Específicas:**
> 
> *   **Em filhotes:** Verminoses (como *Toxocara* e *Giardia*), dieta inadequada, ingestão excessiva de comida ou água, megaesôfago (regurgitação confundida com vômito), ingestão de corpo estranho, obstrução intestinal, envenenamento ([fonte 12]).
> *   **Gatos:** Bolas de pelo (comum em gatos de pelo longo), mudanças na dieta, ingestão de objetos estranhos (fios, elásticos), parasitas intestinais, doenças crônicas (insuficiência renal, doenças hepáticas, diabetes), intoxicação, estresse ([fonte 21]).
> *   **Cães:** Gastrite, alergia ou intoxicação alimentar, úlcera gastro duodenal, insuficiência renal, verminose, giardíase, virose, doenças hepáticas, ingestão de corpos estranhos, pancreatite, cistite, piometra, problemas neurológicos, tumores ([fontes 4, 6]).
> *   **Cães de grande porte:** A torção gástrica é mais comum em cães de grande porte que se alimentam apenas uma vez ao dia ([fonte 24]).
> *   **Envenenamento:** Produtos de limpeza doméstica (não induzir o vômito, mas diluir com água), inseticidas (induzir o vômito com sal grosso e procurar veterinário) ([fonte 7]).
> 
> **Quando Procurar um Veterinário:**
> 
> *   Vômito persistente ou frequente (mais de duas vezes ao dia ou por mais de 24 horas) ([fonte 3]).
> *   Presença de sangue no vômito ([fonte 8]).
> *   Vômito acompanhado de outros sintomas como diarreia, perda de apetite, letargia, febre, dor abdominal, desidratação, mudanças de comportamento ([fonte 3]).
> *   Suspeita de ingestão de substância tóxica ou corpo estranho ([fonte 3]).
> *   Tentativas de vomitar sem sucesso, principalmente em cães ([fonte 3]).
> *   Se o animal for filhote ou idoso ([fonte 3]).
> *   Se o animal tiver alguma doença preexistente ou estiver tomando alguma medicação ([fonte 3]).
> *   Vômitos recorrentes ou acompanhados de outros sintomas (diarreia, febre, letargia) ([fonte 13]).
> *   Se o cão vomitar mais de uma vez por semana ([fonte 12]).
> *   Se o vômito persistir por mais de duas semanas ([fonte 22]).
> 
> **O Que Fazer em Caso de Vômito:**
> 
> *   Retirar a comida por 8 a 12 horas (cães) ou 12-24 horas (adultos), mas manter água fresca disponível ([fonte 2, 3]).
> *   Oferecer água em pequenas quantidades para evitar desidratação. Água de coco pode ajudar ([fonte 3]).
> *   Após o período de jejum, introduzir gradualmente uma dieta leve e de fácil digestão (frango cozido desfiado com arroz branco cozido sem tempero) ([fonte 3]).
> *   Para cachorros, oferecer água gelada ou água de coco ([fonte 10]).
> *   Em caso de vômito com espuma branca, analisar o aspecto para informar o veterinário ([fonte 24]).
> *   Se houver suspeita de envenenamento por inseticida, induzir o vômito com sal grosso e procurar o veterinário ([fonte 7]).
> 
> **O Que Não Fazer:**
> 
> *   Não medicar o animal por conta própria ([fonte 3]).
> *   Não forçar o animal a comer se ele não estiver com apetite ([fonte 3]).
> *   Não ignorar os sinais de alerta ([fonte 3]).
> *   Não provocar vômito sem orientação profissional em caso de suspeita de envenenamento ([fonte 18]).
> 
> **Observações Importantes:**
> 
> *   A cor do vômito pode dar pistas sobre a causa ([fonte 3]):
> 
>     *   Amarelo ou verde: Presença de bile, geralmente indica estômago vazio ([fonte 3]).
>     *   Branco: Excesso de gás no estômago, indigestão ou inflamação ([fonte 3]).
>     *   Marrom: Comida não digerida ou fezes (coprofagia) ([fonte 3]).
>     *   Vermelho: Sangue fresco ([fonte 3]).
>     *   Preto: Sangue digerido, indica sangramento no trato gastrointestinal superior ([fonte 3]).
> *   É importante diferenciar vômito de regurgitação. A regurgitação é passiva, sem esforço abdominal, e geralmente envolve comida não digerida ([fonte 3]).
> *   Vômitos frequentes podem indicar problemas de saúde sérios ([fonte 4, 6]).
> *   Em filhotes, vômito e diarreia podem ser causados por mudanças na dieta, infecções, ingestão de objetos estranhos ou estresse ([fonte 2]).
> *   Se o vômito for intenso, frequente e acompanhado de outros sintomas, procurar ajuda veterinária ([fonte 8]).
> 
> Lembre-se de que essas informações são apenas um guia. Em caso de vômito, consulte sempre um médico veterinário para um diagnóstico preciso e tratamento adequado.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Revisor) ---



> Entendi. Para complementar as informações sobre vômito em cães e gatos, realizei algumas pesquisas no Google para trazer informações mais específicas e atualizadas.
> 
> Com base nas informações encontradas, aqui estão alguns pontos adicionais e complementares sobre o vômito em cães e gatos:
> 
> **Causas Específicas:**
> 
> *   **Em filhotes:** Verminoses (como *Toxocara* e *Giardia*), dieta inadequada, ingestão excessiva de comida ou água, megaesôfago (regurgitação confundida com vômito), ingestão de corpo estranho, obstrução intestinal, envenenamento.
> *   **Gatos:** Bolas de pelo (comum em gatos de pelo longo), mudanças na dieta, ingestão de objetos estranhos (fios, elásticos), parasitas intestinais, doenças crônicas (insuficiência renal, doenças hepáticas, diabetes), intoxicação, estresse.
> *   **Cães:** Gastrite, alergia ou intoxicação alimentar, úlcera gastro duodenal, insuficiência renal, verminose, giardíase, virose, doenças hepáticas, ingestão de corpos estranhos, pancreatite, cistite, piometra, problemas neurológicos, tumores.
> *   **Cães de grande porte:** A torção gástrica é mais comum em cães de grande porte que se alimentam apenas uma vez ao dia.
> *   **Envenenamento:** Produtos de limpeza doméstica (não induzir o vômito, mas diluir com água).
> 
> **Quando Procurar um Veterinário:**
> 
> *   Vômito persistente ou frequente (mais de duas vezes ao dia ou por mais de 24 horas).
> *   Presença de sangue no vômito.
> *   Vômito acompanhado de outros sintomas como diarreia, perda de apetite, letargia, febre, dor abdominal, desidratação, mudanças de comportamento.
> *   Suspeita de ingestão de substância tóxica ou corpo estranho.
> *   Tentativas de vomitar sem sucesso, principalmente em cães.
> *   Se o animal for filhote ou idoso.
> *   Se o animal tiver alguma doença preexistente ou estiver tomando alguma medicação.
> *   Vômitos recorrentes ou acompanhados de outros sintomas (diarreia, febre, letargia).
> *   Se o cão vomitar mais de uma vez por semana.
> *   Se o vômito persistir por mais de duas semanas.
> 
> **O Que Fazer em Caso de Vômito:**
> 
> *   Retirar a comida por 8 a 12 horas (cães) ou 12-24 horas (adultos), mas manter água fresca disponível.
> *   Oferecer água em pequenas quantidades para evitar desidratação.
> *   Após o período de jejum, introduzir gradualmente uma dieta leve e de fácil digestão (frango cozido desfiado com arroz branco cozido sem tempero).
> *   Para cachorros, oferecer água gelada ou água de coco.
> *   Em caso de vômito com espuma branca, analisar o aspecto para informar o veterinário.
> 
> **O Que Não Fazer:**
> 
> *   Não medicar o animal por conta própria.
> *   Não forçar o animal a comer se ele não estiver com apetite.
> *   Não ignorar os sinais de alerta.
> *   Não provocar vômito sem orientação profissional em caso de suspeita de envenenamento.
> 
> **Observações Importantes:**
> 
> *   A cor do vômito pode dar pistas sobre a causa:
> 
>     *   Amarelo ou verde: Presença de bile, geralmente indica estômago vazio.
>     *   Branco: Excesso de gás no estômago, indigestão ou inflamação.
>     *   Marrom: Comida não digerida ou fezes (coprofagia).
>     *   Vermelho: Sangue fresco.
>     *   Preto: Sangue digerido, indica sangramento no trato gastrointestinal superior.
> *   É importante diferenciar vômito de regurgitação. A regurgitação é passiva, sem esforço abdominal, e geralmente envolve comida não digerida.
> *   Vômitos frequentes podem indicar problemas de saúde sérios.
> *   Em filhotes, vômito e diarreia podem ser causados por mudanças na dieta, infecções, ingestão de objetos estranhos ou estresse.
> *   Se o vômito for intenso, frequente e acompanhado de outros sintomas, procurar ajuda veterinária.
> 
> **Importante:** As informações aqui fornecidas são apenas um guia informativo e não substituem a necessidade de uma consulta com um médico veterinário qualificado. A automedicação e a falta de diagnóstico adequado podem levar a sérios riscos para a saúde do seu animal. Em caso de vômito, especialmente se persistente ou acompanhado de outros sintomas, procure sempre um profissional para um diagnóstico preciso e tratamento adequado.


--------------------------------------------------------------
